# Example usage

To use `dbpd` in a project:

In [7]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import dbpd

print(dbpd.__version__)

0.1.0


#### Load the metadata and binaries

In [9]:
import tsdf
import numpy as np
import pandas as pd
from dbpd.constants import DataColumns

metadata_dict = tsdf.load_metadata_from_path('../data/WatchData.IMU.Week0.raw_segment0161_meta.json')

# Retrieve the metadata object we want, using the name of the binary as key
metadata_samples = metadata_dict["WatchData.IMU.Week0.raw_segment0161_samples.bin"]
data = tsdf.load_binary_from_metadata(metadata_samples)
metadata_time = metadata_dict["WatchData.IMU.Week0.raw_segment0161_time.bin"]
time = tsdf.load_binary_from_metadata(metadata_time)


# Print some info
print(f"Data type:\t {data.dtype}")
print(f"Data shape:\t {data.shape}")


Data type:	 int16
Data shape:	 (6757787, 6)


##### Test whether the time data channel is strictly increasing

In [10]:
if not np.all(np.diff(time) > 0):
    print("time_abs_array is not strictly increasing")
    non_increasing_indices = np.where(np.diff(time) <= 0)[0]

    # Printing the problematic parts
    for index in non_increasing_indices[:3]:
        print(f"Non-increasing at index {index}: {time[index]} followed by {time[index + 1]}")
        print(f"time[{index+1}]-time[{index}] = {time[index + 1] - time[index]}\n")

time_abs_array is not strictly increasing
Non-increasing at index 2: 9.853515625 followed by 9.853271484375
time[3]-time[2] = -0.000244140625

Non-increasing at index 3: 9.853271484375 followed by 9.853271484375
time[4]-time[3] = 0.0

Non-increasing at index 4: 9.853271484375 followed by 9.853271484375
time[5]-time[4] = 0.0



#### Setup a preprocessing configuration

In [11]:
ppp = dbpd.PreprocessingPipeline(DataColumns.TIME, 100, 100, True)



#### Resample the data


In [12]:
df = ppp.resample_data(time, data, metadata_samples.scale_factors)
print(df.shape)

ValueError: time_abs_array is not strictly increasing

In [ ]:
df.head(100)

,time,acceleration_x,acceleration_y,acceleration_z,rotation_x,rotation_y,rotation_z
0,0.00,-1.947919,-6.937407,6.655780,0.426829,0.365854,-0.060976
1,0.01,-1.929206,-6.970213,6.617931,0.242929,0.302510,-0.123012
2,0.02,-1.948174,-6.951278,6.556102,0.183581,-0.190263,-0.307185
3,0.03,-1.967239,-6.951788,6.491408,0.242955,-0.549013,-0.428149
4,0.04,-2.000309,-6.965541,6.544878,-0.011529,-0.165813,-0.490807
...,...,...,...,...,...,...,...
95,0.95,-1.953118,-6.922230,6.604785,-0.124503,0.737273,-0.497406
96,0.96,-1.936685,-6.940766,6.622679,-0.376661,0.734967,-0.303513
97,0.97,-1.944537,-6.959815,6.641376,-0.416458,0.705000,-0.239195
98,0.98,-1.953137,-6.940753,6.628472,-0.259175,0.602034,-0.203600
